In [33]:
import datasets
import transformers 
import librosa #load audio files: soundfile package
import jiwer #evaluate fine tuned model using WER metric

In [34]:
from huggingface_hub import notebook_login
notebook_login()

Prepare Data, Tokenizer and Feature Extractor

In [35]:
# load the dataset and look at its struture

from datasets import load_dataset, load_metric

timit = load_dataset("timit_asr", data_dir="/home/ix502iv/Documents/Datasets/timit_large")
print(timit)

Using the latest cached version of the module from /home/ix502iv/.cache/huggingface/modules/datasets_modules/datasets/timit_asr/43f9448dd5db58e95ee48a277f466481b151f112ea53e27f8173784da9254fb2 (last modified on Sun Oct 16 22:10:53 2022) since it couldn't be found locally at timit_asr., or remotely on the Hugging Face Hub.
Using custom data configuration default-43b510b3628aa686
Found cached dataset timit_asr (/home/ix502iv/.cache/huggingface/datasets/timit_asr/default-43b510b3628aa686/0.0.0/43f9448dd5db58e95ee48a277f466481b151f112ea53e27f8173784da9254fb2)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 1680
    })
})


In [36]:
#drop some of the columns : keeps the notebook general

timit = timit.remove_columns(["phonetic_detail","word_detail", "dialect_region","id",
"sentence_type", "speaker_id"])

Transcription of the datasets

In [37]:
from cgitb import html
from datasets import ClassLabel
import random
import pandas as pd
import IPython.display 
from IPython.display import display, HTML

# a function to display some ransdom samples of datasets

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(timit["train"].remove_columns(["file","audio"]))

,text
0,She had your dark suit in greasy wash water all year.
1,She had your dark suit in greasy wash water all year.
2,Carl lives in a lively home.
3,Fights sprang up and were quickly squelched.
4,With this no loyal citizen can quarrel.
5,I gave them several choices and let them set the priorities.
6,New self-deceiving rags are hurriedly tossed on the too-naked bones.
7,The irate actor stomped away idiotically.
8,An adult male baboon's teeth are not suitable for eating shellfish.
9,Did Shawn catch that big goose without help?


In [38]:
#normalizing the text only to have lowercase, and removing the special chars
import re
chars_to_ignore_regex = '[\,\?\!\-\;\:\"]'

def remove_special_chars(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

timit = timit.map(remove_special_chars)

show_random_elements(timit["train"].remove_columns(["file","audio"]))

  0%|          | 0/4620 [00:00<?, ?ex/s]

  0%|          | 0/1680 [00:00<?, ?ex/s]

,text
0,employee layoffs coincided with the company's reorganization.
1,the overweight charmer could slip poison into anyone's tea.
2,differences were related to social economic and educational backgrounds.
3,keep your seats boys i just want to put some finishing touches on this thing.
4,she had your dark suit in greasy wash water all year.
5,count the number of teaspoons of soysauce that you add.
6,the advertising verse of plymouth variety store never changes.
7,the saw is broken so chop the wood instead.
8,don't ask me to carry an oily rag like that.
9,even i occasionally get the monday blues


In [39]:
#mapping function to concatenate all transcriptions inot one long transcription
#transform the entire into a set f chars

def extract_all_chars(batch): #pass the batch: acess all transcriptions at once
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True,
remove_columns=timit.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [40]:
#create a union of all distict letters in the training dataset
# convert the resulting list inot an enumearated dictionary

vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'c': 0,
 'm': 1,
 'f': 2,
 's': 3,
 'b': 4,
 'v': 5,
 'w': 6,
 'k': 7,
 'y': 8,
 '.': 9,
 'p': 10,
 'z': 11,
 "'": 12,
 'r': 13,
 'l': 14,
 'u': 15,
 ' ': 16,
 'd': 17,
 'o': 18,
 'x': 19,
 'i': 20,
 't': 21,
 'h': 22,
 'n': 23,
 'g': 24,
 'e': 25,
 'j': 26,
 'q': 27,
 'a': 28}

In [41]:
vocab_dict["|"] = vocab_dict[" "] #give the spce character a more visible definition
del vocab_dict[" "]

In [42]:
#add a padding token correspoding to CTC's blank token

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD"] = len(vocab_dict)
print(len(vocab_dict))

31


we have a vocabulary list that consists of 31 tokens, therefore, the linear layer that we will
add on top of the pretrained wav2vec2 chackpoint will have anb output dimension of 31

In [44]:
#save the vocabas a json file

import json
with open('vocab.json','w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [45]:
# we use the json file to instantiate an object of the Wav2Vec2CTCTokenizer class

from transformers import Wav2Vec2CTCTokenizer
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]",
                word_delimiter_token="|")


In [46]:
# to re-use the just created tokenizer with the finetuned model of the notebook
repo_name = "wav2vec2-base-timit-demo-vscode"
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/ix502iv/wav2vec2-base-timit-demo-vscode/commit/a0ccb66267cd10681c96bb3c463dcef0c391dbdd', commit_message='Upload tokenizer', commit_description='', oid='a0ccb66267cd10681c96bb3c463dcef0c391dbdd', pr_url=None, pr_revision=None, pr_num=None)